In [1]:
import os
os.environ['TF_ENABLE_TENSORRT']='1'
import tensorflow as tf
from tensorflow import keras
from keras.layers import(Conv2D,MaxPool2D,Conv2DTranspose,concatenate,Dropout,Input,Lambda)
from keras.models import Model
import matplotlib.pyplot as plt

import PIL
from PIL import Image
import numpy as np

from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
print("cuDNN version",tf.config.list_physical_devices('GPU')[0].properties['cudnn_version'])

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)


In [ ]:

data_gen_args={
    'rescale':1.0/255,
    
}


# Create ImageDataGenerator instances for images and masks
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Define the image and mask generators using flow_from_directory
batch_size = 2

# Use the same seed for image and mask generators to ensure they apply the same transformations
seed = 42

train_image_generator = image_datagen.flow_from_directory(
    '/home/carl/Downloads/detection/datasets/trainset_081023/train_images/',
    target_size = (256,256),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    shuffle=False
    
)

train_mask_generator = mask_datagen.flow_from_directory(
    '/home/carl/Downloads/detection/datasets/trainset_081023/train_masks/',
    target_size = (256,256),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    color_mode='grayscale',
    shuffle=False
)

val_image_generator = image_datagen.flow_from_directory(
    '/home/carl/Downloads/detection/datasets/trainset_081023/val_images/',
    target_size = (256,256),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    shuffle=False
)

val_mask_generator = mask_datagen.flow_from_directory(
    
    '/home/carl/Downloads/detection/datasets/trainset_081023/val_masks/',
    target_size = (256,256),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    color_mode='grayscale',
    shuffle=False
)

test_image_generator = image_datagen.flow_from_directory(
    '/home/carl/Downloads/detection/datasets/trainset_081023/test_images/',
    target_size = (256,256),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    shuffle=False
)

test_mask_generator = mask_datagen.flow_from_directory(
    '/home/carl/Downloads/detection/datasets/trainset_081023/test_masks/',
    target_size = (256,256),
    batch_size=batch_size,
    class_mode=None,
    seed=seed,
    color_mode='grayscale',
    shuffle=False
)

# Combine image and mask generators using zip
train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)
test_generator = zip(test_image_generator, test_mask_generator)



In [ ]:
# Checking the outputs of data generators are correct

x= train_image_generator.next()
y=train_mask_generator.next()
for i in range(0,1):
    image = x[i]
    mask=y[i]
    plt.subplot(1,2,1)
    plt.imshow(image[:,:,0],cmap='gray')
    plt.subplot(1,2,2)
    plt.imshow(mask[:,:,0])
    plt.show()

In [ ]:
def unet_model(input_shape):
   
    inputs = Input(shape=input_shape)
    s = Lambda(lambda x: x/255)(inputs)

    #Encoder
    conv0 =Conv2D(16,(3,3),activation="relu",kernel_initializer ="he_normal",padding='same')(s)
    conv0 = Dropout(0.1)(conv0)
    conv0 =Conv2D(16,(3,3),activation="relu",kernel_initializer ="he_normal",padding='same')(conv0)
    pool0= MaxPool2D(pool_size=(2,2))(conv0)


    conv1=Conv2D(32,(3,3),activation="relu",kernel_initializer ="he_normal", padding="same")(pool0)
    conv1 = Dropout(0.1)(conv1)
    conv1=Conv2D(32,(3,3),activation="relu",kernel_initializer ="he_normal", padding="same")(conv1)
    pool1= MaxPool2D(pool_size=(2,2))(conv1)
    

    conv2=Conv2D(64,(3,3),activation="relu",kernel_initializer ="he_normal", padding="same")(pool1)
    conv2 = Dropout(0.1)(conv2)
    conv2=Conv2D(64,(3,3),activation="relu",kernel_initializer ="he_normal", padding="same")(conv2)
    pool2= MaxPool2D(pool_size=(2,2))(conv2)
    

    conv3=Conv2D(128,(3,3),activation="relu",kernel_initializer ="he_normal", padding="same")(pool2)
    conv3 = Dropout(0.1)(conv3)
    conv3=Conv2D(128,(3,3),activation="relu",kernel_initializer ="he_normal", padding="same")(conv3)
    pool3=MaxPool2D(pool_size=(2,2))(conv3)
    

    conv4 =Conv2D(256,(3,3),activation="relu",kernel_initializer ="he_normal", padding="same")(pool3)
    conv4 = Dropout(0.1)(conv4)
    conv4=Conv2D(256,(3,3),activation="relu",kernel_initializer ="he_normal", padding="same")(conv4)
    pool4=MaxPool2D(pool_size=(2,2))(conv4)
    

    #Middle

    convm=Conv2D(512,3,activation="relu",padding="same")(pool4)
    convm=Conv2D(512,3,activation="relu",padding="same")(convm)
    
    #Decoder

    deconv4=Conv2DTranspose(256,(2,2),strides=(2,2),padding="same")(convm)
    uconv4 = concatenate([deconv4,conv4],axis=3)
      
    uconv4=Conv2D(256,(3,3),activation="relu",kernel_initializer ="he_normal",padding="same")(uconv4)
    uconv4 = Dropout(0.1)(uconv4)
    uconv4=Conv2D(256,(3,3),activation="relu",kernel_initializer ="he_normal",padding="same")(uconv4)
    

   
    deconv3=Conv2DTranspose(128,(2,2),strides=(2,2),padding="same")(uconv4)
    uconv3 = concatenate([deconv3,conv3],axis=3)
    
    
    uconv3=Conv2D(128,(3,3),activation="relu",kernel_initializer ="he_normal",padding="same")(uconv3)
    uconv3 = Dropout(0.1)(uconv3)
    uconv3=Conv2D(128,(3,3),activation="relu",kernel_initializer ="he_normal",padding="same")(uconv3)


    deconv2=Conv2DTranspose(64,3,strides=(2,2),padding="same")(uconv3)
    uconv2 = concatenate([deconv2,conv2],axis=3)
    
    
    uconv2=Conv2D(64,(3,3),activation="relu",kernel_initializer ="he_normal",padding="same")(uconv2)
    uconv2 = Dropout(0.1)(uconv2)
    uconv2=Conv2D(64,(3,3),activation="relu",kernel_initializer ="he_normal",padding="same")(uconv2)

    deconv1=Conv2DTranspose(64,3,strides=(2,2),padding="same")(uconv2)
    uconv1 = concatenate([deconv1,conv1],axis=3)
    
    
    uconv1=Conv2D(32,(3,3),activation="relu",kernel_initializer ="he_normal",padding="same")(uconv1)
    uconv1 = Dropout(0.1)(uconv1)
    uconv1=Conv2D(32,(3,3),activation="relu",kernel_initializer ="he_normal",padding="same")(uconv1)

    deconv0=Conv2DTranspose(64,3,strides=(2,2),padding="same")(uconv1)
    uconv0 = concatenate([deconv0,conv0],axis=3)
    
    
    uconv0=Conv2D(16,(3,3),activation="relu",kernel_initializer ="he_normal",padding="same")(uconv0)
    uconv0 = Dropout(0.1)(uconv0)
    uconv0=Conv2D(16,(3,3),activation="relu",kernel_initializer ="he_normal",padding="same")(uconv0)


    

    #output

    output_layer = (Conv2D(1,(1,1),activation="sigmoid"))(uconv0)
    model = Model(inputs=inputs,outputs=output_layer)

    return model

    







In [ ]:
from keras import backend as K

def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

#Dice metric

def dice_metric(y_pred,y_true):
    intersection = K.sum(K.sum(K.abs(y_true*y_pred),axis=-1))
    union = K.sum(K.sum(K.abs(y_true)+K.abs(y_pred),axis=-1))

    return 2*intersection/union

In [ ]:
input_shape = (256,256,3)
model = unet_model(input_shape)
model.compile(optimizer='adam',loss=[jaccard_distance_loss],metrics=['accuracy',dice_metric])
model.summary()

In [ ]:
#model checkpoint

checkpointer=tf.keras.callbacks.ModelCheckpoint('joint_model.h5',verbose=1,save_best_only=True)
callbacks=[
    tf.keras.callbacks.EarlyStopping(patience=4,monitor="val_loss"),
    tf.keras.callbacks.TensorBoard(log_dir='logs')]


num_train_imgs = len(os.listdir('/home/carl/Downloads/detection/datasets/trainset_081023/train_images/train/'))
steps_per_epoch = num_train_imgs//batch_size


history = model.fit(train_generator,steps_per_epoch=steps_per_epoch,epochs=50,validation_data=val_generator,callbacks=callbacks)



In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
#acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
#val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()